In [ ]:
import torch
import torchvision # 데이터셋 불러오는 패키지
from torchvision import transforms
import torch.nn as nn

In [ ]:
#이걸 안하면 train에 overfitting 되는 느낌
transform_train = transforms.Compose([
                                      # transforms.RandomRotation(10),
                                      # transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
                                      # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                                      transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                     ])
                                     

# tensor로 바꿔주고, [-1,1]사이로 정규화 시킴
transform_test = transforms.Compose( [transforms.ToTensor(), 
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# trainset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)

# test 
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
print(trainset.classes)
print(trainset.__getitem__(1))
print(len(trainset))

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
(tensor([[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         ...,
         [ 0.3098,  0.1608, -0.2000,  ..., -0.5137, -0.5608, -0.3098],
         [ 0.3647,  0.1059, -0.6549,  ..., -0.1922, -0.2549, -0.0275],
         [ 0.2863,  0.1059, -0.0980,  ..., -0.3961, -0.4667, -0.2627]],

        [[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         ...,
         [ 0.2471,  0.1373, -0.1843,  ..., -0.4745, -0.5294, -0.2941],
         [ 0.2863,  0.0745, -0.6392,  ..., -0.1529, -0.2314, -0.0353],
         [ 0.2157,  0.0902, -0.0667,  ..., -0.3961, -0.4824, -0.3098]],

        [[-1.0000, -1.00

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ConvNN(nn.Module):
    def __init__(self):
        super(ConvNN, self).__init__()
        # 3 : input channel size  /  48 : output volume size(48개의 filter => ouput의 depth가 될 것), 
        # 3 : kernel(filter) size (3*3)  /  padding = 1 : padding size  /  stride = 1로 default

        # common settings : F = 3, S = 1, P =1 / F = 5, S = 1, P = 2 / F = 5, S = 2, P = ? 

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=48, kernel_size=(3,3), padding=(1,1))
        self.conv2 = nn.Conv2d(in_channels=48, out_channels=96, kernel_size=(3,3), padding=(1,1))
        self.conv3 = nn.Conv2d(in_channels=96, out_channels=192, kernel_size=(3,3), padding=(1,1))
        self.conv4 = nn.Conv2d(in_channels=192, out_channels=256, kernel_size=(3,3), padding=(1,1))
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(in_features=8*8*256, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=64)
        self.Dropout = nn.Dropout(0.25)
        self.fc3 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):         # output 크기
        x = F.relu(self.conv1(x)) # 32 x 32 x 48 > outputsize = (input size - filter size + 2*padding size)/stride + 1 = 32 
        x = F.relu(self.conv2(x)) # 32 x 32 x 96
        x = self.pool(x) # 16 x 16 x 96
        x = self.Dropout(x)
        x = F.relu(self.conv3(x)) # 16 x 16 x 192
        x = F.relu(self.conv4(x)) # 16 x 16 x 256
        x = self.pool(x) # 8 x 8 x 256
        x = self.Dropout(x)
        x = x.view(-1, 8*8*256) # reshape
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.Dropout(x)
        x = self.fc3(x)

        return x

cvnn = ConvNN()
cvnn.cuda()

ConvNN(
  (conv1): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(192, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (Dropout): Dropout(p=0.25, inplace=False)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [ ]:
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cvnn.parameters(), lr=learning_rate)

In [ ]:
from torch.autograd import Variable

epochs = 30 
curr_lr = learning_rate

for epoch in range(epochs):

  tot_loss = 0.0
  correct = 0

  for i, data in enumerate(trainloader, 0) : 

    images, labels = data # batch size만큼 가져올 것
    images, labels = Variable(images).cuda(), Variable(labels).cuda()

    outputs = cvnn(images)
    loss = criterion(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    tot_loss += loss.item()

    _, predicted = torch.max(outputs.data, 1)
    correct += torch.sum(predicted == labels.data)
        
  if (epoch+1) % 20 == 0:
        curr_lr /= 3
        for param_group in optimizer.param_groups:
            param_group['lr'] = curr_lr


  epoch_loss = tot_loss / len(trainloader)
  epoch_acc = correct.float() / len(trainloader)
  
  print("===================================================")
  print("epoch: ", epoch + 1)
  print("train loss: {:.5f}, acc: {:5f}".format(epoch_loss, epoch_acc))


epoch:  1
train loss: 1.72632, acc: 35.536003
epoch:  2
train loss: 1.29686, acc: 52.868004
epoch:  3
train loss: 1.09209, acc: 61.274002
epoch:  4
train loss: 0.94890, acc: 66.602005
epoch:  5
train loss: 0.87283, acc: 69.854004
epoch:  6
train loss: 0.80802, acc: 71.966003
epoch:  7
train loss: 0.76861, acc: 73.422005
epoch:  8
train loss: 0.72654, acc: 75.192001
epoch:  9
train loss: 0.68925, acc: 76.194000
epoch:  10
train loss: 0.67133, acc: 77.192001
epoch:  11
train loss: 0.64231, acc: 78.096001
epoch:  12
train loss: 0.62059, acc: 78.792007
epoch:  13
train loss: 0.60662, acc: 79.446007
epoch:  14
train loss: 0.58408, acc: 80.030006
epoch:  15
train loss: 0.57223, acc: 80.654007
epoch:  16
train loss: 0.55960, acc: 80.858002
epoch:  17
train loss: 0.55212, acc: 81.108002
epoch:  18
train loss: 0.52769, acc: 82.108002
epoch:  19
train loss: 0.52313, acc: 82.078003
epoch:  20
train loss: 0.50912, acc: 82.604004
epoch:  21
train loss: 0.44332, acc: 84.972008
epoch:  22
train loss:

In [ ]:
cvnn.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:

        images, labels = Variable(images).cuda(), labels.cuda()

        outputs = cvnn(images)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()

    print('Accuracy of Test: {} %'.format(100 * correct / total))


Accuracy of Test: 86.25 %
